<a href="https://colab.research.google.com/github/dcmesh/Solar-Eclipse-Data-Analysis/blob/main/gen_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Festival of Frequency Measurement Data Analysis

Colab by Joanna Elia based on Jupyter notebook by Kristina Collins KD8OXT, based on FFM code by Aidan Montare KB3UMD.


## Introduction

First, we need to set up out virtual environment. 

Colab is missing some modules and some need to be updated. 

We also need to mount the drive, which will allow Colab to read and write files within your google drive.

Uncomment the cell below and run it once before recommenting it out. Then restart the runtime (under the Runtime tab) so we can utilize the update modules.

Then run the next cell to import all the modules we installed.

In [ ]:
# run once if using Google Collabratory, this will take a while
# uncomment first run, runtime needs to be restarted to use updated modules
'''
!pip install pyproj
!pip install geopandas
!pip install suntime
!pip install -U plotly

#git libaries used
!pip install --user git+https://github.com/matplotlib/basemap.git
!pip install --user git+https://github.com/HamSCI/eclipse_calculator.git

#to grab Rise set Utils
!wget 'https://raw.githubusercontent.com/dcmesh/Solar-Eclipse-Data-Analysis/main/Rise_set_utils.py'

#to get files- specific to this dataset
!wget https://zenodo.org/record/4400117/files/Freq_full.txt
!wget https://zenodo.org/record/4400117/files/JEF_List.xlsx
!wget https://zenodo.org/record/4400117/files/Vpk_full.txt

#"""

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import matplotlib.dates
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
sns.set(style="darkgrid")
import pyproj
import geopandas
from geopy import distance
from tqdm.notebook import tqdm
from scipy import signal
import itertools
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.mpl_axes import Axes

from Rise_set_utils import midpoint

# For correlations:
import functools

# register progress_apply with pandas
tqdm.pandas(leave=False)

# import necessary libraries for sunrise and sunset calculations:
import datetime
from datetime import timedelta
from suntime import Sun, SunTimeException


# Eclipse Obscuration Calc:
%pylab inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import eclipse_calc

## Switches and Settings

In [ ]:
# turn this on to generate the stackedplots
# generally, leave this off to make the notebook run faster
PLOT_STACKEDPLOTS = True

# Turn this on to cull the data:
CULL_DATA = True

# turn this on to compute time-delayed correlations (which are pretty slow sometimes)
COMPUTE_LARGE_CORRELATIONS = False

# turn this on to load dask distributed client
LOAD_DASK = False

# turn this on to filter data
FILTER_DATA = True

# where the figures will be saved
# must end with /
if FILTER_DATA:
    figures_dir = "drive/Shared drives/Solar Eclipse Data Analysis/filtered_plots/"
else:
    figures_dir = "drive/Shared drives/Solar Eclipse Data Analysis/unfiltered_plots/"


start_day = datetime.date(2021, 6, 19)
day_range = 3
day = datetime.date(2020, 6, 21);

# where data directory is located
data_dir = ""

## Getting Started: Data Import

Let's get some eclipse paths in here, since we'll be needing them soon:

In [ ]:
# Get eclipse traces
from urllib.request import urlopen
import json
with urlopen('https://hamsci.org/sites/default/files/eclipse/experiment/future-eclipses-custom.geojson') as response:
    geojson = json.load(response)

edf=pd.DataFrame()
for i in range(9,17):
    df2=[i, geojson["features"][i]["properties"]['date'], geojson["features"][i]["properties"]['EclipseProperty']]
    df2=pd.DataFrame(df2)
    df2=df2.T
    columns=['id', 'date', 'type']
    if edf.empty:
        edf=df2
    else:
        edf=edf.append(df2)
  
edf.columns=['id', 'date', 'type']


We can map the eclipse traces using either plotly or plotly express:

In [ ]:
# Make an eclipse trace map
fig = px.choropleth_mapbox(edf[0:1], geojson=geojson, color="date",
                           locations="date", featureidkey="properties.date",
                           center={"lat": 20, "lon": 73.7073},
                           mapbox_style="carto-positron", zoom=2)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout_showlegend=False)
fig.show()

fig = px.choropleth(edf, geojson=geojson, color="date",
                           locations="date", featureidkey="properties.date",
                           center={"lat": 20, "lon": 73.7073},
#                            mapbox_style="carto-positron", zoom=2
                   )
fig.update_layout(margin={"r":0,"t":40,"l":40,"b":40})
# fig.update(layout_showlegend=False)
fig.show()

TypeError: ignored

Next we read in the data. (Bonus points if we can figure out how to do this directly from Zenodo, but for now this notebook can link to the directory of the downloaded files.)

In [ ]:
daystring='200'+str(day.month)+str(day.day)

pedigree = pd.read_csv(data_dir + 'JEF Stations.csv')#, dtype={"zID": "object"})
FreqError = pd.read_csv(data_dir + daystring +'_FreqErr.txt', index_col=0, parse_dates=['UTC'])
Amplitude = pd.read_csv(data_dir + daystring + '_Vpk.txt', index_col=0, parse_dates=['UTC'])


## Set Up Data for Analysis

The "pedigree" file contains metadata for each of the stations that collected 10 MHz data.

In [ ]:
# pedigree.loc([2])
pedigree.loc[48,"Callsign"] = '罗 智文'
pedigree.index_col = "Callsign"
pedigree['Status'] = 'keep'
pedigree

We should verify that all the collected datasets have pedigree entries. If FreqError has any columns not accounted for in pedigree, we're going to run into trouble. 

In [ ]:
(FreqError.columns).difference(pedigree.Callsign)
FreqError=FreqError[pd.Index.intersection(FreqError.columns, pedigree.Callsign)]

Add sunrise and sunset times to our table. 

In [ ]:
for i in pedigree["Callsign"].index.values:
#     print(pedigree.loc[i,"Callsign"], pedigree.loc[i,"Latitude"], pedigree.loc[i,"Longitude"])
    sun = Sun(pedigree.loc[i,"Latitude"], pedigree.loc[i,"Longitude"])
#     abd = datetime.date(2020, 6, 21)
    pedigree.loc[i,"Sunrise"] = sun.get_sunrise_time(day)
    pedigree.loc[i,"Sunset"] = sun.get_sunset_time(day)
pedigree

The experimental data from all the stations is organized into tables - one for frequency error, one for amplitude, and so on.

In [ ]:
FreqError.head()
FreqError.columns

#Adding filtering:
filtered_FreqError = FreqError.rolling(window=300, min_periods=1).mean()
unfiltered_FreqError = FreqError
if FILTER_DATA:
    FreqError=filtered_FreqError
    Amplitude = Amplitude.rolling(window=300, min_periods=1).mean()
    
FreqError

In [ ]:
FreqError.G0LHZ

Culling data:

In [ ]:


# pedigree.loc[pedigree["Callsign"] == "NU0C"]["Status"] = "remove"

if CULL_DATA==True:
    for call in ["3V1E", "K6RGI", "K6FOD"]:
        print(call)
        #index = pedigree[pedigree["Callsign"] == call]
        list(pedigree[pedigree["Callsign"] == call].index.values) 
        pedigree.loc[pedigree[pedigree["Callsign"] == call].index.values, "Status"]  = "remove"


Some datasets were culled above.

We discussed which stations appear to have measurement errors on one of our Thursday telecon calls. The notes we generated are in the file `ffm-data-comments.txt`. Kristina also has her own version of this file.

In [ ]:
pedigree[pedigree["Status"] == "remove"]

We can compare that list to the list I made, and indeed, they are the same list of stations.

Rather than outright removing the columns in `FreqError` corresponding to these stations, we'll replace their values with `nan`, the "not a number" value. We'll do this by creating a dataframe of booleans that we can use as a mask: every place with `True` will be kept for analysis, and every place with `False` will be replaced by `nan`.

The advantage of doing this is that we can use the same procedure to mask out parts of a station's record. This will let us remove some of the errors in otherwise fine datasets.

In [ ]:
# dataframe of positions that we will keep
FreqErrorMask = FreqError.applymap(lambda x: True)

Set values as `False` for the stations we are going to remove.

In [ ]:
FreqErrorMask[pedigree[pedigree["Status"] == "remove"]["Callsign"]] = False

And remove the portions of datasets that are otherwise fine:

In [ ]:

# FreqErrorMask.loc["2019-10-01T16:07":"2019-10-01T16:22", "VE6IXD"] = False #3470110
# FreqErrorMask.loc["2019-10-01T15:22":"2019-10-01T16:13", "K0ANS"] = False #3468667
# FreqErrorMask.loc["2019-10-01T23:15":"2019-10-02T00:00", "KA6WKE"] = False #3468667

Here's our mask. It has the same columns and indicies as FreqError.

In [ ]:
FreqErrorMask.head()

We can visualize the mask by creating an image of it. Below, light values (1's) are True, and dark values (0's )are False.

In [ ]:
# sns.heatmap(FreqErrorMask.astype('int'))
# plt.show()

To apply the mask, we use `.where`.

In [ ]:
# FreqError.where(FreqErrorMask).head()

To check that the mask had the intended effect, let's make before and after images showing the location of `nan`'s in FreqError. 

Before:

In [ ]:
sns.heatmap(FreqError.notna().astype('int'))
plt.show()

Note that even the original FreqError dataframe is missing a lot of values. Some of the data may have been eliminated because of the use of matlab's synchronise function to prepare the unified table in the zenodo upload.
https://www.mathworks.com/help/matlab/ref/timetable.synchronize.html

After:

In [ ]:
sns.heatmap(FreqError.where(FreqErrorMask).notna().astype('int'))
plt.show()

In [ ]:
## combined before and after plot
#
#f, axes = plt.subplots(1,2, figsize=(14,6), sharey=True)
#
#sns.heatmap(FreqError.notna().astype('int'), ax=axes[0])
#
#sns.heatmap(FreqError.where(FreqErrorMask).notna().astype('int'), ax=axes[1])
#
#plt.subplots_adjust(top=0.7)
#f.suptitle("Before and after the application of the mask")
#plt.tight_layout()
#plt.show()

Note these kinds of heatmaps also give us another way of visualizing the frequency error data. It doesn't seem as easy to read as the stackedplots, though.

In [ ]:
sns.heatmap(FreqError)
plt.show()

Now let's re-reun the correlations using our masked (we'll call it culled) dataset.

In [ ]:
culled_FreqErrorCorrelations = FreqError.where(FreqErrorMask).corr()

In [ ]:
culled_FreqErrorCorrelations.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(culled_FreqErrorCorrelations)
plt.title("Correlations in (Culled, Raw) Frequency Error")
plt.show()

"They've gone to plaid!" - KC

The Festival of Frequency Measurement was 01 October 2019 0000 to 2359 UTC.

In [ ]:
# date_timestamp = (pd.Timestamp("2019-10-01")
#                     .tz_localize("UTC"))
# date_string = date_timestamp.strftime("%Y-%m-%d")

Since the data is ordered nicely, we could just plot the points in order and ignore any sort of time information. However, it will be much nicer if we use a datetime index for the data.

We now have datetime objects as the index:

In [ ]:
Amplitude.head()


## Bare basics

Let's just plot one of the stations' data. Local sunrise is the dotted line, sunset the solid one.

In [ ]:
Callsign="G0LHZ"
# pandas has some convient plotting functions built in, although datetimes are tricky.
# sr=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunrise"]
# sr=sr.values[0]
# ss=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunset"]
# ss=ss.values[0]

FreqError[Callsign].plot()
# plt.axvline(x=sr, ymin=-1, ymax=1, ls='--', color=[1, 0, 0])
# plt.axvline(x=ss, ymin=-1, ymax=1, ls='-', color=[1, 0, 0])
plt.show()

## Organize Data Geographically

###Using Plotly

From the latitude and longitude data, we can make a map of where the stations are. 

In [ ]:
linklist ='https://qrz.com/db/'+pedigree["Callsign"]
clicklist = str('<')+str('a href=')+str('"\"') + linklist+ str('/\>') + pedigree["Callsign"]
fig = px.scatter_geo(pedigree, "Latitude", "Longitude",
                     color="GPSDO", # which column to use to set the color of markers
                     hover_name=pedigree["Callsign"],#clicklist,#, # column added to hover information
#                      projection = 'albers usa',
                     )
fig.update_layout(title="Stations Submitting Data to the June Eclipse Festival")
fig.show()
fig.write_html(figures_dir +"stationmap.html") #export as html

Plotly Express is incredibly handy, but it only uses dots. For printing in black and white we need a version with different marker types. 

In [ ]:
df = pedigree
df.loc[48,"Callsign"] = 'SWL-02'
# from plotly.validators.scatter.marker import SymbolValidator
WWVlat = [40.678306]
WWVlong = [-105.040889]

markerlist=0+(df['GPSDO'].values == "Yes")
namelist = df['GPSDO'].values

#markerlist
fig = go.Figure(data=go.Scattergeo(
#         projection ='albers usa',
        locationmode = 'ISO-3',
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['Callsign'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol=markerlist,
#             colorscale=[[0, "rgb(166,206,227)"],[0.25, "rgb(31,120,180)"]],
            color = (markerlist+2)
        )
))

# Plot WWV:
fig.add_trace(go.Scattergeo(
        mode="markers+lines",
        lon=WWVlong,
        lat=WWVlat,
        name="WWV",
        marker={'size': 20,
               'color': 'Black',
                'line.color':'Orange',
               'symbol': 'star'}))


fig.update_layout(coloraxis = {'colorscale':'viridis'})
fig.update_layout(
#         title = 'Festival of Frequency Measurement Stations<br>',
        geo = dict(
            scope='world',
#             projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(90, 90, 90)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        )

    )

fig.update_layout(legend_title_text='GPSDO')
fig.update_layout(showlegend=False)

fig.show()

# fig.savefig('printmap.png')
# fig.write_image(figures_dir + "printmap.png")


Let's make a map of the stations with the relevant eclipse trace in it: 

In [ ]:
df = pedigree
# df.loc[48,"Callsign"] = 'SWL-02'
# from plotly.validators.scatter.marker import SymbolValidator
WWVlat = [40.678306]
WWVlong = [-105.040889]

markerlist=0+(df['GPSDO'].values == "Yes")
namelist = df['GPSDO'].values

#fig = px.choropleth(edf[0:1], geojson=geojson, color="date", locations="date", featureidkey="properties.date",center={"lat": 0, "lon": 90})
#                            mapbox_style="carto-positron", zoom=2
                           

# fig = go.Figure(data=go.Choropleth(
#     locations=edf['date'],
# #     z=df['total exports'].astype(float),
# #     locationmode='USA-states',
# #     colorscale='Reds',
# #     autocolorscale=False,
# #     text=df['text'], # hover text
# #     marker_line_color='white', # line markers between states
# #     colorbar_title="Millions USD"
# ))

# fig.update(layout_showlegend=False)
#markerlist

fig.add_trace(go.Scattergeo(
#         projection ='albers usa',
        locationmode = 'ISO-3',
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['Callsign'],
        mode = 'markers',
        name = '',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol=markerlist,
#             colorscale=[[0, "rgb(166,206,227)"],[0.25, "rgb(31,120,180)"]],
            color = (markerlist+2)
        )
))

# Plot WWV:
fig.add_trace(go.Scattergeo(
        mode="markers+lines",
        lon=WWVlong,
        lat=WWVlat,
        name="WWV",
        marker={'size': 20,
               'color': 'Black',
                'line.color':'Orange',
               'symbol': 'star'}))

# Plot BPM:
fig.add_trace(go.Scattergeo(
        mode="markers+lines",
        lon=[109.543036],
        lat=[34.948878],
        name="BPM",
        marker={'size': 20,
               'color': 'Black',
                'line.color':'Orange',
               'symbol': 'star'}))


fig.update_layout(coloraxis = {'colorscale':'viridis'})
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
#         title = 'Festival of Frequency Measurement Stations<br>',
        geo = dict(
            scope='world',
#             projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(90, 90, 90)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        )

    )



fig.update_layout(
    title="June Eclipse Stations",
    legend_title="Legend Title",
#     labels={"date":"GPSDO Stations"}
    
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)

# fig.update_layout(legend_title_text='GPSDO')
fig.update_layout(showlegend=False)
fig.update_layout(height=800, width=1800, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html(figures_dir +"junemap.html") #export as html
# fig.savefig('printmap.png')
# fig.write_image(figures_dir + "junemap.png")

#df.loc[47:48]


In [ ]:
fig1 = px.choropleth(edf[0:1], geojson=geojson, color="date", locations="date", featureidkey="properties.date",center={"lat": 0, "lon": 90})
fig1.show()

###Using Matplotlib

We can then rearrange the stations according to longitude (and latitude, secondarily) and put the culled stations at the end. 

In [ ]:
pedigree.sort_values(by=['Status', 'Longitude', 'Latitude'], ascending=[1, 0, 1], inplace=True)
# FreqError = FreqError[pedigree["Callsign"].astype(str)]

We'll create the stackedplot using matplotlib. First we'll generate a tiny map for each station that we can include in the stackedplot.

In [ ]:
gdf = geopandas.GeoDataFrame(pedigree, geometry=geopandas.points_from_xy(pedigree["Longitude"], pedigree["Latitude"]))
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


In [ ]:
# ax = world.cx[-120:-60,20:70].plot( #North America only
ax = world.cx[-180:180,-90:90].plot(
    color='white', edgecolor='black')

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color='red')

#px.choropleth(edf[0:1], geojson=geojson, color="date", locations="date", featureidkey="properties.date",center={"lat": 0, "lon": 90})
#plt.show()
edf[0:1]

If we want to check any of the parameters of gdf:

In [ ]:
gdf.head()
gdf.dtypes
gdf.shape
pedigree.shape
gdf.total_bounds

##Stacked Plots

###Sample Plot

In [ ]:
Callsign="3V1E"    
# Create a vector of times to evaluate
sTime = datetime.datetime(2020,6,21,0,0)#(2017,8,21,16,0)
eTime = datetime.datetime(2020,6,21,23,0)#(2017,8,21,20,0)
dt    = datetime.timedelta(minutes=5)
q=1

times = [sTime]
while times[-1] < eTime:
  times.append(times[-1]+dt)
  q+=1
#obscuration calcs:
record = gdf[gdf["Callsign"] == Callsign].iloc[0]
obsc  = eclipse_calc.calculate_obscuration(times,record.Latitude,record.Longitude)
x = matplotlib.dates.date2num(times)
yarr = vstack((obsc,))


# pandas has some convient plotting functions built in, although datetimes are tricky.
fig, ax = plt.subplots()
axs = [ax, ax.twinx()]

sr=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunrise"]
sr=sr.values[0]
ss=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunset"]
ss=ss.values[0]

axs[1].plot(FreqError[Callsign])
axs[0].imshow(yarr, extent = [min(x),max(x), -1, 1], aspect = 'auto', cmap= 'binary')
axs[0].axes.get_yaxis().set_visible(False)
fig.autofmt_xdate()


plt.axvline(x=sr, ymin=-1, ymax=1, ls='--', color=[1, 0, 0])
plt.axvline(x=ss, ymin=-1, ymax=1, ls='-', color=[1, 0, 0])

plt.show()

####Using Plotly

Here is the same graph, but with plotly. It runs slower

In [ ]:
px.line(FreqError, x=FreqError.index, y="G0LHZ")

###Stacked Plots

Now let's create that stackedplot

In [ ]:
def suplabel(axis,label,label_prop=None,
             labelpad=5,
             ha='center',va='center'):
    ''' Add super ylabel or xlabel to the figure
    Similar to matplotlib.suptitle
    axis       - string: "x" or "y"
    label      - string
    label_prop - keyword dictionary for Text
    labelpad   - padding from the axis (default: 5)
    ha         - horizontal alignment (default: "center")
    va         - vertical alignment (default: "center")
    
    from https://stackoverflow.com/a/29107972/10008497
    '''
    fig = plt.gcf()
    xmin = []
    ymin = []
    for ax in fig.axes:
        xmin.append(ax.get_position().xmin)
        ymin.append(ax.get_position().ymin)
    xmin,ymin = min(xmin),min(ymin)
    dpi = fig.dpi
    if axis.lower() == "y":
        rotation=90.
        x = xmin-float(labelpad)/dpi
        y = 0.5
    elif axis.lower() == 'x':
        rotation = 0.
        x = 0.5
        y = ymin - float(labelpad)/dpi
    else:
        raise Exception("Unexpected axis: x or y")
    if label_prop is None: 
        label_prop = dict()
    plt.text(x,y,label,rotation=rotation,
               transform=fig.transFigure,
               ha=ha,va=va,
               **label_prop)

Here's the function we'll use to make stackplots. It includes sunrise and sunset times and maps for each station.

In [ ]:
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.mpl_axes import Axes

#debugging previous cell...
def stackedplot2(df, n=None, start=0, sharey=False, amplitude_df=None, mask_df=None, date_limit=None):
    """
    Make a stacked plot for the first n columns in df.
    
    stackedplot2 uses matplotlib, and won't lock up the computer
    forever. Just a while.
    
    Returns a figure object.
    
    start: the first index to plot (default: 0)
    n: the number of columns to plot (default: all the columns)


    """
    if n is None:
        n = df.columns.size
        
    if mask_df is not None:
        fill_df = ~mask_df
    
    fig, axes = plt.subplots(nrows=n, ncols=3, sharey=sharey, 
#                               sharex='col', #comment out to change maps
                             gridspec_kw={'width_ratios': [4, 1, 1]})
    
    world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
    padding = 10

    # comment out these 2 lines:
#     ax = world.cx[-120:-60,20:70].plot(ax=axes[i,1],
#        color='white', edgecolor='black')
    
    i=0
    previous_amplitude_axis = None
    
    for Callsign in tqdm(df.columns[start:(start + n)], desc="Making stackedplot"):
# #     Full world map
        mapbounds = [-180,   180  ,  -90,   90]

        # data
        sr=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunrise"]
        sr=sr.values[0]
#         print('Sunrise for ', Callsign, sr)
        ss=pedigree.loc[pedigree[pedigree["Callsign"] == Callsign].index.values, "Sunset"]
        ss=ss.values[0]
        print('Sunset for ', Callsign, ss)
        
        # ECLIPSE OBSCURATION:
        # Create a vector of times to evaluate
        sTime = datetime.datetime(2020,6,21,0,0)#(2017,8,21,16,0)
        eTime = datetime.datetime(2020,6,22,0,0)#(2017,8,21,20,0)
        dt    = datetime.timedelta(minutes=5)
        q=1

        times = [sTime]
        while times[-1] < eTime:
            times.append(times[-1]+dt)
            q+=1
        #obscuration calcs:
        record = gdf[gdf["Callsign"] == Callsign].iloc[0]
        obsc   = eclipse_calc.calculate_obscuration(times,record.Latitude,record.Longitude)
        x2     = matplotlib.dates.date2num(times)
        yarr   = vstack((obsc,)) 

        # obscuration code needs to be on first axis otherwise it covers the plot
        axes[i,0].imshow(yarr, extent = [min(x2),max(x2), -1, 1], aspect = 'auto', cmap= 'binary')
        axes[i,0].axes.get_yaxis().set_visible(False)
        axes[i,0].axes.get_xaxis().set_visible(False)

        freq_axis = axes[i,0].twinx()
        freq_axis.plot(df[Callsign])
        freq_axis.axvline(x=sr, ymin=-1, ymax=1, ls='--', color=[1, 0, 0])
        freq_axis.axvline(x=ss, ymin=-1, ymax=1, ls='-', color=[1, 0, 0])    
        freq_axis.get_xaxis().set_visible(False) 
        
        axes[i,0].grid(False)
        freq_axis.grid(False)

        #testing this...
        if date_limit is not None:
#             foo = axes[i, 0].set_xlim([737962, 737963])
            foo = axes[i, 0].set_xlim([day, day+timedelta(1)])
#         print(axes[i, 0].get_xlim())
        
        if amplitude_df is not None:
            amplitude_axis = freq_axis.twinx()
            amplitude_axis.plot(amplitude_df[Callsign], 'r')
            if sharey == 'col' and i != 0: 
                #print(i, previous_amplitude_axis, amplitude_axis)
                amplitude_axis.get_shared_y_axes().join(amplitude_axis, previous_amplitude_axis)
                #print(i, amplitude_axis.get_shared_y_axes().get_siblings(amplitude_axis))
            previous_amplitude_axis = amplitude_axis
            amplitude_axis.grid(False)
        
        if mask_df is not None:
            limits = axes[i,0].get_ylim()
            axes[i,0].fill_between(df.index, limits[0], limits[1], where=fill_df[Callsign],
                                   color="white", alpha=1)
            # see https://github.com/matplotlib/matplotlib/issues/3872
            #collection = collections.BrokenBarHCollection.span_where(
            #    df.index, ymin=0, ymax=1, where=mask_df[zID], facecolor='green', alpha=0.5)
            #ax.add_collection(collection)
        # TODO make time scale visible

        # location map
        world.cx[-180:180,-90:90].plot(ax=axes[i,1],
            color='white', edgecolor='black')
        gdf[gdf["Callsign"] == Callsign].plot(ax=axes[i,1], color='red')
#         axes[i,1].set_xlim(mapbounds[0] - padding, mapbounds[2] + padding)
#         axes[i,1].set_ylim(mapbounds[1] - padding, mapbounds[3] + padding)
        axes[i,1].tick_params(labelleft=False, labelbottom=False)
        
        # station info
        record = gdf[gdf["Callsign"] == Callsign].iloc[0]
        text = ("Callsign: {callsign}\n"
#                 "Status: {status}\n"
                "Latitude: {latitude:.4}\n"
                "Longitude: {longitude:.4}\n"
                "GPSDO: {gpsdo}").format(callsign=record.Callsign,
                                          status=record.Status,
                                          latitude=record.Latitude,
                                          longitude=record.Longitude,
                                          gpsdo=record["GPSDO"])
        axes[i, 2].text(0, 0, text)
        axes[i, 2].axis('off')
        i+=1
        
    #axes[n - 1, 0].get_xaxis().set_major_locator(matplotlib.dates.HourLocator(interval=3))
    
    axes[n - 1, 0].get_xaxis().set_major_formatter(matplotlib.dates.DateFormatter('%m/%d %H:%M'))
    plt.setp(axes[n-1,0].get_xticklabels(), rotation=30, ha="right")
        
    axes[n-1, 0].get_xaxis().set_visible(True) #testing this...
    
    axes[0, 0].set_title("Submitted Frequency Data")
    axes[0, 1].set_title("Map")
    axes[0, 2].set_title("Info")
    
    axes[n - 1, 0].set_xlabel("Time (UTC)")
    
    #axes[n//2, 0].set_ylabel("Frequency Deviation", rotation="vertical")

    suplabel("y", "Frequency Deviation (Hz)")
    
    #fig.text(0, 0, "Frequency Deviation (Hz)", ha="center", va="center", rotation="vertical")
    
    return fig
    gdf[gdf["Callsign"] == Callsign].iloc[0]

Let's test this out with a small set of plots. 

In [ ]:
fig = stackedplot2(FreqError, n=5, start=5, mask_df=FreqErrorMask, amplitude_df=Amplitude, date_limit='None')
print('Figure created.')
fig.set_size_inches(12,6)
fig.savefig(figures_dir + daystring + "_sample-stackedplot.png", bbox_inches="tight", pad_inches=0)
print('Figure written.')



Here's where we generate the split stackedplot so it can be printed easily on a single page. This cell will take a while to run:

In [ ]:
if PLOT_STACKEDPLOTS:
    # make the two-columns layout for paper
    first_in_column_two = int(np.ceil(FreqError.columns.size/2))
    sizes = [first_in_column_two, FreqError.columns.size - first_in_column_two]
    print(sizes)
    
    fig1 = stackedplot2(FreqError, n=sizes[1], start=1, mask_df=FreqErrorMask, amplitude_df=Amplitude, date_limit='None')# date_limit=1)
    fig1.set_size_inches(12,24)
#     fig.tight_layout()
    fig1.savefig(figures_dir + daystring+"_stackedplot-split-1.png", bbox_inches="tight", pad_inches=0)
    
    fig2 = stackedplot2(FreqError, n=sizes[1], start=first_in_column_two, mask_df=FreqErrorMask, amplitude_df=Amplitude, date_limit='None')#, date_limit=1)
    fig2.set_size_inches(12,24)
    #fig2.savefig('paper-stackedplot-split-2.png')
    #fig.tight_layout()
    fig2.savefig(figures_dir + daystring + "_stackedplot-split-2.png", bbox_inches="tight", pad_inches=0)
#     FreqError=qux

##Eclipse Map

This code that creates an Eclipse Map is separate from the rest of the notebook. It is https://github.com/HamSCI/eclipse_calculator/blob/master/eclipse_maps.py with different dates. 

If you only want to create the eclipse map, run these two cells, make sure to uncomment the installs in the first cell, if the code in the Intro wasn't run.

###TODO change eclipse_maps.py to a function with parameters of start and end time and interval

In [ ]:
"""
#git libaries used
!pip install --user git+https://github.com/matplotlib/basemap.git
!pip install --user git+https://github.com/HamSCI/eclipse_calculator.git

#mount drive to get data from and save files to
from google.colab import drive
drive.mount('/content/drive')
#"""

In [ ]:
#!/usr/bin/env python3
import os
import datetime
from collections import OrderedDict
import multiprocessing

import numpy as np
import pandas as pd

import matplotlib as mpl
mpl.use("Agg")
from matplotlib import pyplot as plt

from astropy.coordinates import EarthLocation

import eclipse_calc

def location_dict(precision=2,height=0.):
    gs_grid     = eclipse_calc.locator.gridsquare_grid(precision=precision).flatten()
    ll_grid     = eclipse_calc.locator.gridsquare2latlon(gs_grid)
    lats,lons   = ll_grid

    dd              = OrderedDict()
    dd['grid']      = gs_grid
    dd['lat']       = lats
    dd['lon']       = lons
    dd['height']    = np.ones(lats.shape)*height
#    dd['loc']   = [EarthLocation.from_geodetic(lon,lat,height) for lat,lon in zip(lats,lons)]
    return dd

def plot_eclipse_dict(run_dict):
    return plot_eclipse(**run_dict)

def plot_eclipse(date,loc_dict,region='world',cmap=mpl.cm.gray_r,output_dir='output'):
    """
    region: 'us' or 'world"
    height: [km]
    """
    # Define output paths.
    date_str    = date.strftime('%Y%m%d_%H%M')
    fname       = '{!s}_{!s}km_eclipseObscuration'.format(date_str,int(height/1000))
    fpath       = os.path.join(output_dir,fname)
    print('Processing {!s}...'.format(fpath))

    # Set up data dictionary.
    dd          = OrderedDict()
    dd['grid']  = loc_dict['grid']
    dd['lat']   = loc_dict['lat']
    dd['lon']   = loc_dict['lon']
    dd['height']= loc_dict['height']
#    locs        = loc_dict['loc']

    # Eclipse Magnitude
#    dd['obsc']  = np.array([eclipse_calc.calculate_obscuration(date,loc=loc) for loc in locs])
    dates       = np.array(len(dd['lat'])*[date])
    dd['obsc']  = eclipse_calc.calculate_obscuration(dates,dd['lat'],dd['lon'],height=dd['height'])
#    dd['obsc']  = dd['lat'] # Useful for debugging plotting.

    # Store into dataframe.
    df          = pd.DataFrame(dd)
    df          = df.set_index('grid')

    # Save CSV Datafile.
    csv_path    = fpath+'.csv'
    with open(csv_path,'w') as fl:
        fl.write('# Solar Eclipse Obscuration file for {!s}\n'.format(date))
    df.to_csv(csv_path,mode='a')

    # Plot data.
    map_prm = {}
    if region == 'world':
        # Map boundaries for the world
        map_prm['llcrnrlon'] = -180.
        map_prm['llcrnrlat'] = -90
        map_prm['urcrnrlon'] = 180.
        map_prm['urcrnrlat'] = 90.
    else:
        # Map boundaries for the United States
        map_prm['llcrnrlon'] = -130.
        map_prm['llcrnrlat'] =   20.
        map_prm['urcrnrlon'] =  -60.
        map_prm['urcrnrlat'] =   55.

    vmin        = 0.
    vmax        = 1.
    cbar_ticks  = np.arange(0,1.1,0.1)

    fig         = plt.figure(figsize=(12,10))
    ax          = fig.add_subplot(111)
    hmap        = eclipse_calc.maps.HamMap(date,date,ax,show_title=False,**map_prm)
    hmap.overlay_gridsquares(label_precision=0,major_style={'color':'0.8','dashes':[1,1]})
    hmap.overlay_gridsquare_data(dd['grid'],dd['obsc'],vmin=vmin,vmax=vmax,cbar_ticks=cbar_ticks,
                zorder=5,cmap=cmap,cbar_shrink=0.5,cbar_label='Obscuration')

    title       = '{!s} Height: {!s} km'.format(date.strftime('%d %b %Y %H%M UT'),height/1000.)
    fontdict    = {'size':'x-large','weight':'bold'}
    hmap.ax.text(0.5,1.025,title,fontdict=fontdict,transform=ax.transAxes,ha='center')
    fig.tight_layout()
    fig.savefig(fpath+'.png',bbox_inches='tight')

    plt.close(fig)

    return fpath

if __name__ == '__main__':
    output_dir  = 'drive/My Drive/output'
    eclipse_calc.gen_lib.clear_dir(output_dir,php=True)

    sDate   = datetime.datetime(2020,12,14,14)
    eDate   = datetime.datetime(2020,12,14,18,30)
#    sDate   = datetime.datetime(2017,8,21,18)
#    eDate   = datetime.datetime(2017,8,21,19)
    dt      = datetime.timedelta(minutes=5)

    precision   = 4
    height      = 300e3

    loc_dict    = location_dict(precision=precision,height=height)

    run_list    = []
    cDate       = sDate
    while cDate < eDate:
        tmp = OrderedDict()
        tmp['date']         = cDate
        tmp['loc_dict']     = loc_dict
        tmp['output_dir']   = output_dir
        run_list.append(tmp)
        cDate   += dt

#    # Single Processor
#    for run_dict in run_list:
#        fpath = plot_eclipse_dict(run_dict)

    with multiprocessing.Pool() as pool:
        pool.map(plot_eclipse_dict,run_list)
